# bundling model scaler dan classification

In [35]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd

X_scaled = pd.read_csv('./ready-train.csv')
y_one_hot = pd.read_csv('./ready-y.csv')

display(X_scaled.head(1), y_one_hot.head(1))

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
0,-0.481475,-0.200421,-0.280055,-0.711333,-0.013272,0.236831,-0.984159,0.143833,0.273099,-1.183088,...,0.570091,-0.715013,-0.266184,0.569352,-0.740524,-0.1082,0.607564,-0.70986,-0.013558,0.625611


,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,64,65
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_one_hot, test_size=0.2, random_state=42
)

X_train.shape[1]

63

In [37]:
# Muat model scaler dari file .h5
scaler_model = load_model('standard_scaler_model.h5')

total_class = y_one_hot.shape[1]

# Definisikan input layer
input_layer = Input(shape=(X_train.shape[1],), name='input')
scaled_input = scaler_model(input_layer)

# Tambahkan layer neural network
x = Dense(64, activation="relu")(scaled_input)
x = Dense(32, activation="relu")(x)
# x = Dropout(0.5)(x)

output_layer = Dense(total_class, activation="softmax")(x)

# Buat model yang menggabungkan scaler dan neural network
combined_model = Model(inputs=input_layer, outputs=output_layer)

combined_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
history = combined_model.fit(
    # X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test)
    X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test)
)

Epoch 1/10
551/551 [==============================] - 2s 2ms/step - loss: 1.6792 - accuracy: 0.7565 - val_loss: 0.3107 - val_accuracy: 0.9489
Epoch 2/10
551/551 [==============================] - 1s 1ms/step - loss: 0.2988 - accuracy: 0.9455 - val_loss: 0.2865 - val_accuracy: 0.9496
Epoch 3/10
551/551 [==============================] - 1s 1ms/step - loss: 0.2572 - accuracy: 0.9498 - val_loss: 0.2194 - val_accuracy: 0.9566
Epoch 4/10
551/551 [==============================] - 1s 1ms/step - loss: 0.2133 - accuracy: 0.9565 - val_loss: 0.2455 - val_accuracy: 0.9519
Epoch 5/10
551/551 [==============================] - 1s 1ms/step - loss: 0.2324 - accuracy: 0.9509 - val_loss: 0.2221 - val_accuracy: 0.9571
Epoch 6/10
551/551 [==============================] - 1s 1ms/step - loss: 0.2153 - accuracy: 0.9537 - val_loss: 0.2049 - val_accuracy: 0.9580
Epoch 7/10
551/551 [==============================] - 1s 1ms/step - loss: 0.2082 - accuracy: 0.9544 - val_loss: 0.1899 - val_accuracy: 0.9637
Epoch 

In [38]:
model_name = "bundling-scale-and-calssification-19am"
model_path = f"{model_name}.h5"
combined_model.save(model_path)
print(combined_model.summary())

# Memuat kembali model tanpa custom_objects
model_load = tf.keras.models.load_model(model_path)

# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(combined_model)
tflite_model = converter.convert()

# Save the TFLite model
with open(f"{model_name}.tflite", "wb") as f:
    f.write(tflite_model)

# Load the TFLite model and check for errors
interpreter = tf.lite.Interpreter(model_path=f"{model_name}.tflite")
interpreter.allocate_tensors()

print("Model loaded successfully with no errors.")

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 63)]              0         
                                                                 
 sequential_28 (Sequential)  (None, 66)                8610      
                                                                 
 dense_6 (Dense)             (None, 64)                4288      
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 66)                2178      
                                                                 
Total params: 17156 (67.02 KB)
Trainable params: 17028 (66.52 KB)
Non-trainable params: 128 (512.00 Byte)
_________________________________________________________________
None


c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


INFO:tensorflow:Assets written to: C:\Users\whisn\AppData\Local\Temp\tmp9jdduq_f\assets


INFO:tensorflow:Assets written to: C:\Users\whisn\AppData\Local\Temp\tmp9jdduq_f\assets


Model loaded successfully with no errors.


In [39]:
model_load = tf.keras.models.load_model(model_path)
model_load.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 63)]              0         
                                                                 
 sequential_28 (Sequential)  (None, 66)                8610      
                                                                 
 dense_6 (Dense)             (None, 64)                4288      
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 66)                2178      
                                                                 
Total params: 17156 (67.02 KB)
Trainable params: 17028 (66.52 KB)
Non-trainable params: 128 (512.00 Byte)
_________________________________________________________________


In [40]:
import joblib
label_encoder = joblib.load("label_encoder.pkl")

In [41]:
y_pred = model_load.predict(X_test)
y_pred_label = label_encoder.inverse_transform(y_pred.argmax(axis=1))
print(y_pred_label[:5])

y_test_array = y_test.to_numpy()  # Konversi DataFrame ke array NumPy
y_true_label = label_encoder.inverse_transform(y_test_array.argmax(axis=1))
print(y_true_label[:5])

138/138 [==============================] - 0s 876us/step
['S' 'Hari ini' 'Malam' 'I' 'Sekarang']
['A' 'Hari ini' 'Malam' 'I' 'Sekarang']


In [44]:
eval_df = pd.DataFrame(y_test)
eval_df["true_kategorik"] = label_encoder.inverse_transform(y_test_array.argmax(axis=1))
eval_df["predicted_kategorik"] = y_pred_label
print(eval_df.shape)

(4404, 68)


In [46]:
print(eval_df[eval_df["true_kategorik"] != eval_df["predicted_kategorik"]].shape)
eval_df = eval_df[['true_kategorik', 'predicted_kategorik']]
eval_df[eval_df["true_kategorik"] != eval_df["predicted_kategorik"]].head(20)

(191, 68)


,true_kategorik,predicted_kategorik
369,A,S
16948,Senang,Kerja
199,A,P
12829,N,M
3222,D,P
6848,Ibu,Makan
14064,P,B
1045,B,E
247,A,X
4966,G,F
